In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [21]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data.parquet')
test_day.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.280902,985,ABEV3.SA,NaN,0.520882,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.266663,227,ABEV3.SA,-0.026404,0.518242,-1.0,0.026404,0.0,0.0,NaN,NaN,NaN,NaN,-0.000059,-0.000033,NaN,-227
2,2000-01-07,0.494478,0.494478,0.494478,0.494478,0.266663,151,ABEV3.SA,0.000000,0.515865,-1.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,-0.000149,-0.000081,NaN,-378
3,2000-01-10,0.494478,0.494478,0.494478,0.494478,0.266663,1516,ABEV3.SA,0.000000,0.513727,1.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,-0.000264,-0.000143,NaN,-1894
4,2000-01-11,0.494478,0.494478,0.494478,0.494478,0.266663,3791,ABEV3.SA,0.000000,0.511802,1.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,-0.000397,-0.000218,NaN,-5685


In [22]:
test_day['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [23]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.tail(15)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
6257,2024-12-03,13.25,13.88,13.19,13.85,13.126506,72892200,ABEV3.SA,0.600000,12.783034,NaN,0.000000,0.60,81.461744,12.25,13.88,32.701480,-67.298520,-0.053987,-0.067615,0.011485,-3108331558
6258,2024-12-04,13.90,14.06,13.78,13.93,13.202327,50659000,ABEV3.SA,0.080000,12.897731,NaN,0.000000,0.08,82.329095,12.25,14.06,35.786222,-64.213778,-0.031705,-0.060433,0.022567,-3057672558
6259,2024-12-05,14.05,14.46,14.00,14.42,13.666730,38794000,ABEV3.SA,0.490000,13.049958,NaN,0.000000,0.49,86.720182,12.25,14.46,36.197180,-63.802820,-0.001743,-0.048695,0.034087,-3018878558
6260,2024-12-06,14.30,14.53,14.15,14.22,13.477178,46449200,ABEV3.SA,-0.200000,13.166962,NaN,0.200000,0.00,77.634647,12.25,14.53,40.217634,-59.782366,0.031084,-0.032739,0.044019,-2972429358
6261,2024-12-09,14.22,14.33,14.14,14.21,13.467700,25321800,ABEV3.SA,-0.010000,13.271266,NaN,0.010000,0.00,77.168206,12.25,14.53,44.792363,-55.207637,0.064771,-0.013237,0.052304,-2947107558
6262,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.371139,NaN,0.000000,0.06,78.079955,12.25,14.53,49.172779,-50.827221,0.098392,0.009089,0.060742,-2924165058
6263,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.414025,NaN,0.470000,0.00,57.372365,12.25,14.53,51.053749,-48.946251,0.127033,0.032677,0.065881,-2875014458
6264,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.382623,NaN,0.700000,0.00,39.410551,12.25,14.53,49.676446,-50.323554,0.145520,0.055246,0.062161,-2924631358
6265,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.352361,NaN,0.020000,0.00,39.007935,12.25,14.53,48.349152,-51.650848,0.155932,0.075383,0.054316,-2955499258
6266,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.301125,NaN,0.240000,0.00,34.174082,12.25,14.53,46.101957,-53.898043,0.158225,0.091951,0.040530,-2987594758


In [27]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2024-11-14'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.head(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
6246,2024-11-14,12.69,12.90,12.61,12.64,11.979713,38027500,ABEV3.SA,-0.089999,12.687755,NaN,0.089999,0.00,49.308863,12.26,13.09,51.536783,-48.463217,-0.061281,-0.035330,-0.013126,-3142666658
6247,2024-11-18,12.48,12.67,12.38,12.42,11.771206,37251000,ABEV3.SA,-0.220000,12.660980,NaN,0.220000,0.00,38.275438,12.26,13.09,48.310814,-51.689186,-0.065434,-0.041350,-0.013778,-3179917658
6248,2024-11-19,12.45,12.60,12.42,12.43,11.780684,28564100,ABEV3.SA,0.010000,12.637882,NaN,0.000000,0.01,38.991434,12.26,13.09,45.527926,-54.472074,-0.069784,-0.047037,-0.014365,-3208481758
6249,2024-11-21,12.31,12.48,12.30,12.39,11.742773,18400700,ABEV3.SA,-0.040000,12.613094,NaN,0.040000,0.00,37.010044,12.26,12.90,55.170926,-44.829074,-0.074375,-0.052505,-0.013821,-3226882458
6250,2024-11-22,12.49,12.70,12.47,12.68,12.017624,24116600,ABEV3.SA,0.290000,12.619784,NaN,0.000000,0.29,55.799500,12.26,12.90,56.216341,-43.783659,-0.076590,-0.057322,-0.011444,-3202765858
6251,2024-11-25,12.59,12.69,12.54,12.54,11.884937,22015800,ABEV3.SA,-0.140000,12.611806,NaN,0.140000,0.00,47.848994,12.26,12.90,54.969707,-45.030293,-0.078089,-0.061475,-0.008685,-3224781658
6252,2024-11-26,12.59,12.71,12.54,12.61,11.951281,16924800,ABEV3.SA,0.070000,12.611625,NaN,0.000000,0.07,51.810260,12.26,12.90,54.941482,-45.058518,-0.078389,-0.064858,-0.007042,-3241706458
6253,2024-11-27,12.63,12.74,12.51,12.55,11.894415,48572000,ABEV3.SA,-0.059999,12.605463,NaN,0.059999,0.00,48.190096,12.26,12.90,53.978588,-46.021412,-0.078221,-0.067531,-0.006580,-3290278458
6254,2024-11-28,12.55,12.80,12.32,12.40,11.752250,40805900,ABEV3.SA,-0.150001,12.584917,NaN,0.150001,0.00,40.106251,12.26,12.90,50.768222,-49.231778,-0.078838,-0.069792,-0.008520,-3331084358
6255,2024-11-29,12.33,12.73,12.25,12.73,12.065012,62396000,ABEV3.SA,0.330000,12.599425,NaN,0.000000,0.33,57.993557,12.25,12.90,53.757699,-46.242301,-0.077265,-0.071287,-0.006962,-3268688358


In [15]:
# Carregar o modelo salvo
rand_frst_clf = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model.pkl')

In [28]:
Features = test_day_AMBEV[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent,r_percent,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
6246,49.308863,51.536783,-48.463217,-0.013126,-0.061281,-0.035330,-3142666658
6247,38.275438,48.310814,-51.689186,-0.013778,-0.065434,-0.041350,-3179917658
6248,38.991434,45.527926,-54.472074,-0.014365,-0.069784,-0.047037,-3208481758
6249,37.010044,55.170926,-44.829074,-0.013821,-0.074375,-0.052505,-3226882458
6250,55.799500,56.216341,-43.783659,-0.011444,-0.076590,-0.057322,-3202765858


In [29]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

# Visualizar o DataFrame com a nova coluna
test_day_AMBEV.head()

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
6246,2024-11-14,12.69,12.90,12.61,12.64,11.979713,38027500,ABEV3.SA,-0.089999,12.687755,-1.0,0.089999,0.00,49.308863,12.26,13.09,51.536783,-48.463217,-0.061281,-0.035330,-0.013126,-3142666658
6247,2024-11-18,12.48,12.67,12.38,12.42,11.771206,37251000,ABEV3.SA,-0.220000,12.660980,1.0,0.220000,0.00,38.275438,12.26,13.09,48.310814,-51.689186,-0.065434,-0.041350,-0.013778,-3179917658
6248,2024-11-19,12.45,12.60,12.42,12.43,11.780684,28564100,ABEV3.SA,0.010000,12.637882,1.0,0.000000,0.01,38.991434,12.26,13.09,45.527926,-54.472074,-0.069784,-0.047037,-0.014365,-3208481758
6249,2024-11-21,12.31,12.48,12.30,12.39,11.742773,18400700,ABEV3.SA,-0.040000,12.613094,1.0,0.040000,0.00,37.010044,12.26,12.90,55.170926,-44.829074,-0.074375,-0.052505,-0.013821,-3226882458
6250,2024-11-22,12.49,12.70,12.47,12.68,12.017624,24116600,ABEV3.SA,0.290000,12.619784,1.0,0.000000,0.29,55.799500,12.26,12.90,56.216341,-43.783659,-0.076590,-0.057322,-0.011444,-3202765858


In [30]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)